In [215]:
import pandas as pd
import numpy as np
import random
import scipy.stats as st
import datetime as dt

In [293]:
def one_myks_test(d,col1,col2,dist='binomial'):
    #sort dataset
    d1 = d[col1].sort_values().to_numpy()
    d2 = d[col2].sort_values().to_numpy()
    #get all x observations and sort again
    #p = 1- (np.sum(d**2)/np.sum(d)) + np.mean(d)
    xbar = np.mean(d2)
    xstd = np.std(d2)
    if dist == 'binomial':
        p = 1 - (xstd / xbar)
        n = np.round(xbar / p)
    elif dist == 'poisson':
        lamb = xbar
    elif dist == 'geometric':
        p = 1/xbar
    else: 
        "Invalid distribution"
    
    #initialize empty lists to store eCDF of dataset and CDF of distribution testing against
    ecdf = []
    #loop through each x observation and take the sum of the indicator RV and multiply by the sample size
    for t in d1:
        ecdf.append((1/len(d1))*sum(d1 <= t))
    
    if dist == 'binomial':
        dist_cdf = st.binom.cdf(d1, n, p)
    elif dist == 'poisson':
        dist_cdf = st.poisson.cdf(d1, lamb)
    elif dist == 'geometric':
        dist_cdf = st.geom.cdf(d1, p)
    else: 
        "Invalid distribution"
        
    #create a results dataframe
    res = pd.DataFrame()
    res['ecdf'] = ecdf
    res['dist_cdf'] = dist_cdf
    
    #remove duplicates
    res.drop_duplicates().reset_index(drop=True)
    
    #find the absolute difference between the two sample eCDFs
    res['abs_diff'] = abs(res['ecdf']-res['dist_cdf'])

    #find the max of the absolute differences, this is the KS statistic
    diff = max(res['abs_diff'])

    return {"KS Statistic": diff}

In [200]:
def two_myks_test(d,col1,col2):
    #sort individual datasets
    d1 = d[col1].sort_values().to_numpy()
    d2 = d[col2].sort_values().to_numpy()
    
    #get all x observations and sort again
    total = np.concatenate((d1,d2))
    total.sort()
    
    #initialize empty lists to store eCDF of respective dataset
    ecdf1 = []
    ecdf2 = []
    
    #loop through each x observation and take the sum of the indicator RV and multiply by the sample size
    for t in total:
        ecdf1.append((1/len(d1))*sum(d1 <= t))
        ecdf2.append((1/len(d2))*sum(d2 <= t))

    #create a results dataframe
    res = pd.DataFrame()
    res['total'] = total
    res['ecdf1'] = ecdf1
    res['ecdf2'] = ecdf2
    
    #remove duplicates
    res.drop_duplicates().reset_index(drop=True)
    
    #find the absolute difference between the two sample eCDFs
    res['abs_diff'] = abs(res['ecdf1']-res['ecdf2'])

    #find the max of the absolute differences, this is the KS statistic
    d = max(res['abs_diff'])

    return {"KS Statistic": d}

In [211]:
def myperm_test(d,col1,col2,num_it):
    #Permutation Test
    #Step 1 :Compute T Obs = abs difference between two sample means
    d1_bar = np.mean(d[col1])
    d2_bar = np.mean(d[col2])
    
    Tobs = abs(d1_bar - d2_bar)
    
    combined_list = np.concatenate((d[col1].to_numpy(),d[col2].to_numpy()))
    
    n = len(d[col1])
    m = len(d[col2])
    
    Ti = []
    
    for i in range(0,num_it):
        rnd_perm = np.random.permutation(combined_list)
        xi = rnd_perm[:n]
        yi = rnd_perm[n:]

        d1i_bar = np.mean(xi)
        d2i_bar = np.mean(yi)
        ti = abs(d1i_bar - d2i_bar)
        Ti.append(ti)
        
    pval = sum( i > Tobs for i in Ti)*(1/num_it)
    
    return {"p-value": pval}

In [216]:
df = pd.read_csv('3.2_linear_interpolation.csv')
df['DATE']= pd.to_datetime(df['DATE'])

In [58]:
def pct_chg_calc(df,colname):
    #takes the dataset and the respective column and calculates the difference between two adjacent rows and divides by the previous
    return df[colname].diff()/df[colname].shift()

In [113]:
def norm_df(df):
    #create new dataframe to store results
    normdf = pd.DataFrame()
    #take the date column from the original
    normdf[df.columns[0]] = df[df.columns[0]].copy()
    #loop through each column with the percent change function defined above
    for column in df.columns[1:]:
        newcol = "{}_per_change".format(column)
        normdf[newcol] = pct_chg_calc(df,column)
        #delete first row in results dataframe as there is no previous value to calculate a percent change frome
        normdf = normdf.iloc[1:]
    return normdf

In [218]:
normdf = norm_df(df)

In [240]:
normdfsub = normdf[(normdf['DATE'] >= '2018-01-01') & (normdf['DATE'] <= '2020-12-31')]
dfsub = df[(df['DATE'] >= '2018-01-01') & (df['DATE'] <= '2020-12-31')]

In [296]:
dists = ['binomial','geometric','poisson']
for d in dists:
    print(one_myks_test(dfsub,'Rent_of_Primary_residence','Monthly_Housing_Cost',dist=d))

{'KS Statistic': 0.9722222222222222}
{'KS Statistic': 0.7155928534558861}
{'KS Statistic': 0.9722221630498638}


In [237]:
two_myks_test(normdfsub,'Monthly_Housing_Cost_per_change','Rent_of_Primary_residence_per_change')

{'KS Statistic': 0.3611111111111111}

In [238]:
myperm_test(normdfsub,'Monthly_Housing_Cost_per_change','Rent_of_Primary_residence_per_change',1000)

{'p-value': 0.228}